In [1]:
!pip install keras==2.1.6 hyperopt==0.1 networkx==1.11
#Note: This works with Python 3.6.8. For Python 3.7, use hyperopt 0.1.2

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv 

from hyperopt import hp
from sklearn.model_selection import train_test_split

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [2]:
import time
from os import listdir
import sys

#Append local dir to path to import custom modules.
local_dir = ""
sys.path.append(local_dir + "/../models")

#Sometimes the following modues won't be imported in Colab. Try uploading a new module and adding/removing sys.path.append to get it to work.
from simpleCNN import simple_cnn_model
from deepNN import deep_nn_model
from processModel import MetaModel

#load MNIST
from keras.datasets import mnist
from keras import backend as K

import keras
from keras import layers
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras.preprocessing import image
from keras import regularizers
from keras.optimizers import Adam, Nadam, Adagrad, Adadelta
from keras.activations import softmax
from keras.losses import sparse_categorical_crossentropy

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

# getting rid of annoying warnings.
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [4]:
#Preprocess the data

# input image dimensions
img_rows, img_cols = 28, 28

#Number of possible target values for y.
classes = 10
#File to log model results
model_res_file = 'modelResults1.csv'

#Load and setup the meta-model.
meta_model = MetaModel(local_dir, model_res_file)

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

def plot_train_val(history):
  """Method to plot the out of a call to keras model.fit.
  """
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
#Get the cross validation data.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (48000, 28, 28, 1)
48000 train samples
12000 validation samples
10000 test samples


In [0]:
#Code to write the columns for the modelResults csv. This info will be logged
#after the models are trained and evaluated.
with open(local_dir + '/' + model_res_file, mode='w') as file:
  writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  writer.writerow(['networkType', 'trainAcc', 'trainTime (s)', 'testAcc', 'layers', 'epochs', 'activations', 'optim', 'learning_rate','reg_param', 'dropout_prob', 'normTech' , 'misc'])

In [5]:
#A simple single stage convolutional network.

#The neural network architectures will be built using combinations of the following parameters.
params = {'classes' : [classes],
          'activation' : ['prelu', 'relu', 'tanh'],
          'batch_size': [64],
          'epochs': [50],
          'optimizer': ['rmsprop', 'adadelta', 'adagrad'],
          'loss': ['sparse_categorical_crossentropy'],
          'reg_param': [0],
          'callbacks' : [[keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')]]}

#Hyperopt will optimize the following parameters in the given ranges.
search_space = {'lr': hp.loguniform('lr', -6, -4), 'dropout_prob' : hp.uniform('dropout_prob', 0.0, 0.6)}

#The call to process_model will compile, train and evaulate the above models and optimize the hyper parameters.
#The output will be saved to a file and the output details will be recorded in the model_res_file. 
model_name = "simpleCNN"
meta_model.max_evals = 3
model_results = meta_model.process_model(params, search_space, simple_cnn_model, model_name, [X_train, X_val, X_test, y_train, y_val, y_test], 
              [model_name, 0, 0, 0, 'layers', 'epochs', 'activation', 'optimizer', 'lr','reg_param', 'dropout_prob', np.NaN , np.NaN])

#Now lets plot the results.
#history = model_results[0][0]
#plot_train_val(history)

W0810 03:08:17.972030 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:508: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 03:08:17.991092 140433291319168 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3144: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0810 03:08:18.007775 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3837: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 03:08:18.042321 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:168:

{'classes': 10, 'activation': 'prelu', 'batch_size': 64, 'epochs': 50, 'optimizer': 'rmsprop', 'loss': 'sparse_categorical_crossentropy', 'reg_param': 0, 'callbacks': [<keras.callbacks.EarlyStopping object at 0x7fb8fd6cfe48>], 'dropout_prob': 0.07489338120701139, 'lr': 0.007202338280154828}


W0810 03:08:18.558062 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1801: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0810 03:08:18.651619 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3661: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0810 03:08:18.699115 140433291319168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:757: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0810 03:08:18.819005 140433291319168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions fo

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 10s 212us/step - loss: 4.6239 - acc: 0.6894 - val_loss: 0.1152 - val_acc: 0.9667
Epoch 2/50
48000/48000 [==============================] - 7s 146us/step - loss: 0.0812 - acc: 0.9756 - val_loss: 0.0644 - val_acc: 0.9832
Epoch 3/50
48000/48000 [==============================] - 7s 146us/step - loss: 0.0615 - acc: 0.9814 - val_loss: 0.0573 - val_acc: 0.9833
Epoch 4/50
48000/48000 [==============================] - 7s 144us/step - loss: 0.0544 - acc: 0.9838 - val_loss: 0.0638 - val_acc: 0.9818
Epoch 5/50
48000/48000 [==============================] - 7s 144us/step - loss: 0.0490 - acc: 0.9844 - val_loss: 0.0551 - val_acc: 0.9841
Epoch 6/50
48000/48000 [==============================] - 7s 144us/step - loss: 0.0437 - acc: 0.9863 - val_loss: 0.0607 - val_acc: 0.9841
Epoch 7/50
12000/12000 [==============================] - 1s 97us/step


W0810 03:09:12.197845 140433291319168 nn_ops.py:4224] Large dropout rate: 0.555952 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


...

{'classes': 10, 'activation': 'prelu', 'batch_size': 64, 'epochs': 50, 'optimizer': 'rmsprop', 'loss': 'sparse_categorical_crossentropy', 'reg_param': 0, 'callbacks': [<keras.callbacks.EarlyStopping object at 0x7fb8fd6cfe48>], 'dropout_prob': 0.544312716464993, 'lr': 0.006583617073750991}


W0810 03:09:50.502453 140433291319168 nn_ops.py:4224] Large dropout rate: 0.544313 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 8s 160us/step - loss: 4.8996 - acc: 0.6728 - val_loss: 4.8579 - val_acc: 0.6857
Epoch 2/50
48000/48000 [==============================] - 7s 151us/step - loss: 0.3081 - acc: 0.9541 - val_loss: 0.0723 - val_acc: 0.9771
Epoch 3/50
48000/48000 [==============================] - 7s 153us/step - loss: 0.0725 - acc: 0.9776 - val_loss: 0.0903 - val_acc: 0.9742
Epoch 4/50
48000/48000 [==============================] - 7s 152us/step - loss: 0.0648 - acc: 0.9803 - val_loss: 0.0667 - val_acc: 0.9810
Epoch 5/50
48000/48000 [==============================] - 7s 150us/step - loss: 0.0596 - acc: 0.9813 - val_loss: 0.0785 - val_acc: 0.9778
Epoch 6/50
10000/10000 [==============================] - 1s 99us/step


W0810 03:10:42.555482 140433291319168 nn_ops.py:4224] Large dropout rate: 0.556546 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Loss = 0.06667118707485498
Test Accuracy = 0.9802
{'classes': 10, 'activation': 'prelu', 'batch_size': 64, 'epochs': 50, 'optimizer': 'adadelta', 'loss': 'sparse_categorical_crossentropy', 'reg_param': 0, 'callbacks': [<keras.callbacks.EarlyStopping object at 0x7fb8fd6cfe48>], 'dropout_prob': 0.5565458516255086, 'lr': 0.007009178982546022}
Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 8s 164us/step - loss: 1.7464 - acc: 0.4464 - val_loss: 0.9525 - val_acc: 0.6860
Epoch 2/50
48000/48000 [==============================] - 7s 151us/step - loss: 0.7488 - acc: 0.7562 - val_loss: 0.6125 - val_acc: 0.7976
Epoch 3/50
48000/48000 [==============================] - 7s 152us/step - loss: 0.5418 - acc: 0.8267 - val_loss: 0.4930 - val_acc: 0.8417
Epoch 4/50
48000/48000 [==============================] - 7s 151us/step - loss: 0.4556 - acc: 0.8570 - val_loss: 0.4213 - val_acc: 0.8666
Epoch 5/50
48000/48000 [==============================] 

In [15]:
#A basic single layer neural network

#Reshape the data for processing by neural nets
X_train = X_train.reshape(X_train.shape[0], img_rows * img_cols)
X_val = X_val.reshape(X_val.shape[0],  img_rows * img_cols)
X_test = X_test.reshape(X_test.shape[0],  img_rows * img_cols)

params = {'layers': [[500]],
          'classes' : [classes],
          'activation' : ['sigmoid', 'relu', 'tanh'],
          'batch_size': [64],
          'epochs': [50],
          'optimizer': ['sgd', 'adadelta', 'adagrad'],
          'loss': ['sparse_categorical_crossentropy'],
          'reg_param': [0],
          'callbacks' : [[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')]]}

search_space = {'lr': hp.loguniform('lr', -5, -3), 'dropout_prob' : hp.uniform('dropout_prob', 0.0, 0.5)}

#Compile the model
model_name = "simpleNN"
meta_model.max_evals = 3
meta_model.verbose = 0
model_results = meta_model.process_model(params, search_space, deep_nn_model, model_name, [X_train, X_val, X_test, y_train, y_val, y_test], 
              [model_name, 0, 0, 0, 'layers','epochs', 'activation', 'optimizer', 'lr','reg_param', 'dropout_prob', np.NaN , np.NaN])

#Now lets plot the results.
#history = model_results[0][0]
#plot_train_val(history)

100%|██████████| 3/3 [04:39<00:00, 88.41s/it, best loss: 0.0881148839861465]


In [0]:
#A basic multi-layer neural network

#Reshape the data for processing by neural nets
X_train = X_train.reshape(X_train.shape[0], img_rows * img_cols)
X_val = X_val.reshape(X_val.shape[0],  img_rows * img_cols)
X_test = X_test.reshape(X_test.shape[0],  img_rows * img_cols)

params = {'layers': [[1000, 500, 500, 500]],
          'classes' : [classes],
          'activation' : ['sigmoid', 'relu', 'tanh'],
          'batch_size': [64],
          'epochs': [50],
          'optimizer': ['sgd', 'adadelta', 'adagrad'],
          'loss': ['sparse_categorical_crossentropy'],
          'reg_param' : [0],
          'callbacks' : [[keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')]]}

search_space = {'lr': hp.loguniform('lr', -6, -4), 'dropout_prob' : hp.uniform('dropout_prob', 0.0, 0.6)}

#Compile the model
model_name = "deepNN"
meta_model.max_evals = 3
meta_model.verbose = 0
model_results = meta_model.process_model(params, search_space, deep_nn_model, model_name, [X_train, X_val, X_test, y_train, y_val, y_test], 
              [model_name, 0, 0, 0, 'layers', 'epochs', 'activation', 'optimizer', 'lr','reg_param', 'dropout_prob', np.NaN , np.NaN])

#Now lets plot the results.
#history = model_results[0][0]
#plot_train_val(history)

In [7]:
#Display the model results after sorting by test accuracy.
import pandas as pd
data = pd.read_csv(local_dir + '/' + model_res_file)
data['testAccPerT'] = 100 * data['testAcc'] / data['trainTime (s)']
data = data.sort_values('testAcc', ascending = False)
data.head(200)

,networkType,trainAcc,trainTime (s),testAcc,layers,epochs,activations,optim,learning_rate,reg_param,dropout_prob,normTech,misc,testAccPerT
50,simpleCNN,0.991625,128.453790,0.9862,NaN,50,relu,adagrad,0.005078,0,0.213305,NaN,NaN,0.767747
47,simpleCNN,0.991896,127.392681,0.9860,NaN,50,prelu,adagrad,0.005030,0,0.161162,NaN,NaN,0.773985
48,simpleCNN,0.994375,65.084619,0.9837,NaN,50,relu,rmsprop,0.009270,0,0.037894,NaN,NaN,1.511417
53,simpleCNN,0.987083,159.142339,0.9823,NaN,50,tanh,adagrad,0.004153,0,0.132492,NaN,NaN,0.617246
45,simpleCNN,0.985812,41.673621,0.9802,NaN,50,prelu,rmsprop,0.003117,0,0.555952,NaN,NaN,2.352087
49,simpleCNN,0.982187,366.359355,0.9800,NaN,50,relu,adadelta,0.010895,0,0.212429,NaN,NaN,0.267497
46,simpleCNN,0.982083,417.585114,0.9798,NaN,50,prelu,adadelta,0.010060,0,0.054030,NaN,NaN,0.234635
32,simpleNN,0.998708,82.006778,0.9794,200,50,relu,adagrad,0.021706,0,0.002780,NaN,NaN,1.194292
17,deepNN,0.999250,89.528609,0.9791,1000-500-500-500,50,tanh,adagrad,0.009503,0,0.031815,NaN,NaN,1.093617
41,simpleNN,0.996854,60.324102,0.9788,500,50,relu,adagrad,0.044943,0,0.189095,NaN,NaN,1.622569
